In [ ]:
from obspy.core import Stream, read, UTCDateTime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
path_to_files = '/Users/sydneydybing/StrainProject/M6_500km_sel/StrainData_sel/'

# Reading in the p-wave pick times, modified from the original Snuffler file to add other information
picks = pd.read_csv('/Users/sydneydybing/StrainProject/M6_500km_sel/SeismicData_sel/Picks/CSVs/allpicks.csv')

print(picks)

In [ ]:
# Pulling the individual columns out of the pick file
eqnum = picks.eqnum.values
date = picks.date.values
time = picks.time.values
sta = picks.sta.values
mag = picks.mag.values

In [ ]:
# Grabbing only the station names from the longer string generated by Snuffler
stas = [x[3:7] for x in sta]

# Grabbing only the years from the longer date string generated by Snuffler
years = [y[0:4] for y in date]

# Creating a list of identifiers for each earthquake, formatted based on the 
# file structure I'm using. 

quakes = []

for i in range(len(picks)):
        
    quake = str(eqnum[i]) + '_' + str(years[i]) + '_' + str(mag[i])
    quakes.append(quake)

In [ ]:
# Looping through the different events and stations based on the times in the
# pick file, the station names I extracted, and the event IDs I generated.

for i, j, k in zip(range(len(picks)), range(len(stas)), range(len(quakes))):
    
    quake = quakes[k]
    sta = stas[j]
 
    # Grabbing the p-wave pick times from the pick file, and altering them
    # slightly into UTCDateTime format to read the files with Obspy.
    
    pt = str(date[i]) + 'T' + str(time[i])
    ptime = UTCDateTime(pt)

    # Establishing the start time for the data - 10 seconds before the p-wave.

    stime = ptime - 10
    
    # Reading in the RMS strain files and trimming to the chosen start time, and
    # adding zeros to pad the data if necessary.
    
    RMS = read(path_to_files + quake + '/' + sta + '_RMS.mseed')
    RMS_trim = RMS.trim(starttime = stime, pad = True, fill_value = 0)
    
    # Extracting times for plotting from the number of points and the sampling
    # rate in the strain metadeta.
    
    times = range(RMS_trim[0].stats.npts)
    times = np.asarray(times)/(RMS_trim[0].stats.sampling_rate)
    
    # Extracting just the RMS data, now trimmed properly, and saving it to a 
    # new miniSEED file.
    
    RMS_data = RMS_trim[0].data
    RMS_trim[0].stats.channel = 'tBSR'
    # RMS_trim.write('/Users/sydneydybing/StrainProject/M6_500km_sel/StrainData_sel/Trimmed/' + quake + '/' + sta + '_tRMS' + '.mseed', format='MSEED')
    
    # Plotting the aligned earthquakes.
    
    plt.plot(times, RMS_data*10**6, label = quake)
    #plt.legend()
    plt.xlim(0,30)
    plt.ylim(0,5)
    plt.xlabel('Time (s)')
    plt.ylabel('RMS Microstrain ($10^{-6}$ m/m)')